New Data Specifications:
- bacteria marked according to its center
- 60 Frames per Second
- all bacteria according to Bhas is labelled - motile or non motile
- about 100 frames per video

In [1]:
%config Completer.use_jedi = False

In [2]:
import zipfile
import openpyxl
import os
import tqdm
import csv
import cv2
import shutil
import PIL
import glob
import pandas as pd
import numpy as np
from natsort import natsorted
from PIL import Image

# Loading New Data

Change only data_dir and video_num and update in video mapping file

In [3]:
target_data_sub_dir = "/data/medha/Bacteria/Data/liquid/"
video_map_path = "/data/medha/Bacteria/Data/videomap.txt"

# data_labels_exist_easy = 0
# data_labels_exist_hard = 1
# data_labels_exist_veryhard = 1

In [4]:
def create_video(data_dir):
    # choose codec according to format needed
    fourcc = cv2.VideoWriter_fourcc(*'mp4v') 
    #print(data_dir)
    img_sample = cv2.imread(os.path.join(data_dir,"images/0.tif"))
    #print(img_sample.shape)
    height, width, channels = img_sample.shape
    
    video = cv2.VideoWriter(data_dir + 'video.mp4', fourcc, 1, (width, height))
    #data_dir = "./Data/video3/"
    image_dir = os.path.join(data_dir, "images")
    for frame in natsorted(os.listdir(image_dir)):
        #print(frame)
        img = cv2.imread(os.path.join(image_dir, frame))
        video.write(img)

    cv2.destroyAllWindows()
    video.release()

In [5]:
#iterate through video1,2,4
#create videos for all frame sets- 1-1,1-2..2-1..4-2
# data_path = target_data_sub_dir
# test_video = [f"video{57}"]
# for video in natsorted(test_video):
#     if not video.startswith('.') and os.path.isdir(os.path.join(data_path,video))==True:
#         for minivideo in natsorted(os.listdir(os.path.join(data_path,video))):
#             if not minivideo.startswith('.') and os.path.isdir(os.path.join(data_path,video,minivideo)) == True:
#                 #print(minivideo)
#                 create_video(os.path.join(data_path,video,minivideo))
    
    

In [5]:
def get_background(file_path, mean=True, sample=False):
    if not os.path.exists(file_path):
        create_video(file_path.rsplit("/",1)[0] +"/frame1")
    cap = cv2.VideoCapture(file_path)
    #print(cap.read())
    # we will randomly select 50 frames for the calculating the median
    #frame_indices = cap.get(cv2.CAP_PROP_FRAME_COUNT) * np.random.uniform(size=20)
    frame_indices = list(range(0,int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) ))
    print(len(frame_indices))
    # we will store the frames in array
    frames = []
    for idx in frame_indices:
        # set the frame id to read that particular frame
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        ret, frame = cap.read()
        #print(ret)
        frames.append(frame)
    if mean:
         # calculate the mean
        background_frame = np.mean(frames, axis=0).astype(np.uint8)
    else:
        # calculate the median
        background_frame = np.median(frames, axis=0).astype(np.uint8)
    if sample==True:
        background_frame = cv2.imread("./Control_2_b0t5306c0x0-660y0-492.tiff")
        #background_frame = cv2.imread("./RBS 2_1_b0t2791c0x0-660y0-492.tiff")
    return background_frame

In [8]:
get_background(f"{target_data_sub_dir}/video{257}/frame1video.mp4").shape

100


(492, 660, 3)

In [8]:
# from PIL import Image
# from numpy import *
# background_img = PIL.Image.fromarray(get_background(f"{target_data_sub_dir}/video{29}/frame1video.mp4"))
# frame_img = Image.open(f"{target_data_sub_dir}/video{29}/frame1/images/13.tif")
# #Image.fromarray(get_background(f"{target_data_sub_dir}/video{30}/frame1video.mp4"), "RGB").show()


In [9]:
# frame_diff = PIL.ImageChops.subtract(frame_img, background_img)
# PIL.ImageChops.subtract(background_img, frame_img)

# Feature Generation Code for Training

In [9]:
# def get_absolute_consec_frame_diff_feature(video_path, max_consec_frame_diff=True, num_prior_frames=1):
#     cap = cv2.VideoCapture(video_path)
#     frames = []
#     differences = []
#     prev = None
#     while True:
#         ret, frame = cap.read()
#         #print(frame)
#         if not ret:
#             break
#         if prev is None:
#             #print(prev)
#             prev = frame
#             continue
#         #computing frame diff between current and one previous
#         consecutive_diff = np.abs(frame - prev)
#         frames.append(frame) #creating frame list
#         #creating consecutive frame diff list
#         differences.append(consecutive_diff)
#     #creating consecutive frame diff features by taking the max at every pixel along the frame diff list
#     if max_consec_frame_diff:
#         max_abs_consec_diff_feature = np.max(differences, axis=0)
#     else:
#          max_abs_consec_diff_feature = np.min(differences, axis=0)
#     return max_abs_consec_diff_feature

def get_absolute_consec_frame_diff_feature(video_path, max_consec_frame_diff=True, num_prior_frames=1):
    cap = cv2.VideoCapture(video_path)
    frames = []
    differences = []
    prev_frames = []
    while True:
        ret, frame = cap.read()
        #print(frame)
        if not ret:
            break
        #storing old frames till prior frame number
        if num_prior_frames>0:
            prev_frames.append(frame)
            num_prior_frames -= 1
            continue
        #retrieving the previous xth frame
        prev = prev_frames[0]
        prev_frames.pop(0)
        #computing frame diff between current and one previous
        consecutive_diff = np.abs(frame - prev)
        frames.append(frame) #creating frame list
        #creating consecutive frame diff list
        differences.append(consecutive_diff)
    #creating consecutive frame diff features by taking the max at every pixel along the frame diff list
    if max_consec_frame_diff:
        max_abs_consec_diff_feature = np.max(differences, axis=0)
    else:
         max_abs_consec_diff_feature = np.min(differences, axis=0)
    return max_abs_consec_diff_feature
    
def get_diff_from_absolute_consec_frame_diff_feature(image_path, frame_diff_feature, frame=False):
    if frame==False:
        image = PIL.Image.open(image_path).convert('L')
    if frame == True:
        #print("getting diff from frame")
        image = Image.fromarray(image_path).convert('L')
    L = image.getchannel(0)
    frame_diff_feature = PIL.Image.fromarray(frame_diff_feature).convert('L')
    frame_diff_feature.getbands()
    L1 = frame_diff_feature.getchannel(0)
    diff = PIL.ImageChops.difference(L, L1)
    return diff

def get_absolute_all_frame_diff_feature(video_path, max_feature=True):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while True:
        ret, frame = cap.read()
        #print(frame)
        if not ret:
            break
        frames.append(frame)
    
    features = []
    count = 0
    for index in tqdm.tqdm(range(len(frames))):
        differences = []
        count+=1
        frame = frames[index]
        for index_1 in (range(len(frames))):
            if index == index_1:
                #print(count)
                continue
            frame1 = frames[index_1]
            differences.append(np.abs(frame - frame1))
        if max_feature:
            max_diff_feature = np.max(differences, axis=0)
            features.append(max_diff_feature)
        else:
            #print("min")
            min_diff_feature = np.min(differences, axis=0)
            features.append(min_diff_feature)
    return features    

In [10]:
# def gen_dense_optical_flow_data(method, video_path, params=[], to_gray=False, median=False, median_frame=None, num_frames_prior=1):
#     frames_optical_flow = []
#     frames_orignal = []
#     # Read the video and first frame
#     cap = cv2.VideoCapture(video_path)
#     #print("fps",cap.get(cv2.CAP_PROP_FPS))
#     ret, old_frame = cap.read()

#     if median == True:
#         old_frame = median_frame
#         old_frame = cv2.cvtColor(old_frame, cv2.COLOR_GRAY2BGR)
#     # crate HSV & make Value a constant
#     hsv = np.zeros_like(old_frame)
#     hsv[..., 1] = 255

#     # Preprocessing for exact method
#     if to_gray:
#         old_frame = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
#     while True:
#         # Read the next frame
#         ret, new_frame = cap.read()
#         frame_copy = new_frame
#         if not ret:
#             break

#         # Preprocessing for exact method
#         if to_gray:
#             new_frame = cv2.cvtColor(new_frame, cv2.COLOR_BGR2GRAY)

#         # Calculate Optical Flow
#         flow = method(old_frame, new_frame, None, *params)

#         # Encoding: convert the algorithm's output into Polar coordinates
#         mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
#         # Use Hue and Value to encode the Optical Flow
#         hsv[..., 0] = ang * 180 / np.pi / 2
#         hsv[..., 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)

#         # Convert HSV image into BGR for demo
#         bgr = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
#         #plt.imshow(frame_copy)
#         #plt.imshow(bgr)
#         # plt.imshow(v,cmap='gray', vmin=0, vmax=255)
#         # plt.show()
    
#         # Update the previous frame
#         old_frame = new_frame
#         if median == True:
#             old_frame = median_frame
#         frames_orignal.append(frame_copy)
#         frames_optical_flow.append(bgr)
#     return frames_orignal, frames_optical_flow

In [11]:
#updated to include optical floiw computtaion from x frames prior
def gen_dense_optical_flow_data(method, video_path, params=[], to_gray=False, median=False, median_frame=None, num_frames_prior=1):
    frames_optical_flow = []
    frames_orignal = []
    # Read the video and first x frames
    cap = cv2.VideoCapture(video_path)
    #print("fps",cap.get(cv2.CAP_PROP_FPS))
    old_frames = []
    for i in range(num_frames_prior):
        ret, old_frame = cap.read()
        # crate HSV & make Value a constant
        hsv = np.zeros_like(old_frame)
        hsv[..., 1] = 255
        if to_gray:
            old_frame = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
        old_frames.append(old_frame)

    #to compute optical flow from the median background    
    if median == True:
        old_frame = median_frame
        old_frame = cv2.cvtColor(old_frame, cv2.COLOR_GRAY2BGR)
        
  

    
    while True:
        # Read the next frame
        ret, new_frame = cap.read()
        frame_copy = new_frame
        if not ret:
            break
        
        old_frame = old_frames[0]
        if median == True:
            old_frame = median_frame

        # Preprocessing for exact method
        if to_gray:
            new_frame = cv2.cvtColor(new_frame, cv2.COLOR_BGR2GRAY)
                
            
        # Calculate Optical Flow
      
        flow = method(old_frame, new_frame, None, *params)

        # Encoding: convert the algorithm's output into Polar coordinates
        mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
        # Use Hue and Value to encode the Optical Flow
        hsv[..., 0] = ang * 180 / np.pi / 2
        hsv[..., 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)

        # Convert HSV image into BGR for demo
        bgr = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
        #plt.imshow(frame_copy)
        #plt.imshow(bgr)
        # plt.imshow(v,cmap='gray', vmin=0, vmax=255)
        # plt.show()
    
        # Update the previous frame
        old_frames.append(new_frame)
        old_frames.pop(0)
        frames_orignal.append(frame_copy)
        frames_optical_flow.append(bgr)
    return frames_orignal, frames_optical_flow

In [12]:
def get_background_diff_feature(image_path, background):
    image = PIL.Image.open(image_path).convert('L')
    L = image.getchannel(0)
    background_img = PIL.Image.fromarray(background)
    background_img.getbands()
    L1 = background_img.getchannel(0)
    diff = PIL.ImageChops.difference(L, L1)
    return diff
    
def get_prev_frame_diff_feature(image_path, prev_image_path):
    image = PIL.Image.open(image_path).convert('L')
    L = image.getchannel(0)
    image = PIL.Image.open(prev_image_path).convert('L')
    L1 = image.getchannel(0)
    diff = PIL.ImageChops.difference(L, L1)
    return diff

def create_feature_image(image_path, background, prev_image_path, prev_image_path2=None):
    prev_image_diff = get_prev_frame_diff_feature(image_path, prev_image_path)
    image = PIL.Image.open(image_path).convert('L')
    L = image.getchannel(0)
    if prev_image_path2 is None:
        background_diff = get_background_diff_feature(image_path, background)
        newImagediff = PIL.Image.merge("RGB", [L, background_diff, prev_image_diff])
    if prev_image_path2 is not None:
        prev_image_diff2 = get_prev_frame_diff_feature(image_path, prev_image_path2)
        newImagediff = PIL.Image.merge("RGB", [L, prev_image_diff2, prev_image_diff])
        
    return newImagediff

def create_feature_image_optical_flow(frame, optical_flow, pure=False, background=None, optical_flow2=None, final_channel=False):
    frame = Image.fromarray(frame).convert('L')
    L = frame.getchannel(0)
    flow = PIL.Image.fromarray(optical_flow)
    flow.save("flow.png")
    hsv_optical_flow = cv2.cvtColor(optical_flow, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv_optical_flow)
    flow = PIL.Image.fromarray(v)
    flow.save("flow_hsv.png")
    
    #print(v.shape)
    last_channel = np.zeros([v.shape[0], v.shape[1]],dtype=np.uint8)
    last_channel[:] = 255
    v = Image.fromarray(v).convert('L')
    v = v.getchannel(0)
    
    
    #print(last_channel)
    last_channel = Image.fromarray(last_channel).convert('L')
    last_channel = last_channel.getchannel(0)

    gray_optical_flow = Image.fromarray(cv2.cvtColor(optical_flow, cv2.COLOR_BGR2GRAY)).convert('L').getchannel(0)
    
    feature_image = PIL.Image.merge("RGB", [L, v, last_channel])
    feature_image = PIL.Image.merge("RGB", [L, gray_optical_flow, last_channel])
    
    if pure == False:
        if background is None:
            gray_optical_flow2 = Image.fromarray(cv2.cvtColor(optical_flow2, cv2.COLOR_BGR2GRAY)).convert('L').getchannel(0)
            feature_image = PIL.Image.merge("RGB", [L, gray_optical_flow, gray_optical_flow2])
            #print(here)
        else:
            if final_channel == False:
                background_img = PIL.Image.fromarray(background)
                L1 = background_img.getchannel(0)
                feature_image = PIL.Image.merge("RGB", [L, v, L1]) #-->org code
                # diff = PIL.ImageChops.difference(L, L1)
                # feature_image = PIL.Image.merge("RGB", [L, v, diff]) #-->up code
                #print("here adding background")
                #print("background")
            if final_channel==True:
                #print("final channel tru, just adding diff")
                feature_image = PIL.Image.merge("RGB", [L, v, background]) 
            
       
        
    # print(feature_image.size)
    # print(feature_image.mode)
        
    return feature_image

In [13]:
def save_data(image_dest, count, feature_image, frame_num, data_dir, video, minivideo, count_test, count_train, train):
    #save image feature
    feature_image.save(image_dest +"images_feature/"+str(count)+".tif")

    text_file = str(frame_num) +".txt"
    annotations_easy_source = os.path.join(data_dir, video, minivideo, "annotations_easy",
                                           text_file)
    annotations_easy_hard_source = os.path.join(data_dir, video, minivideo, 
                                            "annotations_easy_hard", text_file)
    
    annotations_very_hard_source = os.path.join(data_dir, video, minivideo, "annotations_veryhard",
                                           text_file)
    annotations_easy_hard_veryhard_source = os.path.join(data_dir, video, minivideo, 
                                            "annotations_easy_hard_veryhard", text_file)


    shutil.copy(annotations_easy_source, image_dest +"annotation_easy/" +str(count)+".txt")
    shutil.copy(annotations_easy_hard_source, image_dest +"annotation_easy_hard/" 
                + str(count)+".txt")
    if os.path.exists(annotations_very_hard_source):
        shutil.copy(annotations_very_hard_source, image_dest +"annotation_veryhard/" +str(count)+".txt")
    if os.path.exists(annotations_easy_hard_veryhard_source):
        shutil.copy(annotations_easy_hard_veryhard_source, image_dest +"annotation_easy_hard_veryhard/" 
                + str(count)+".txt")
    #create hard only annotation
    text_file_easy_hard = open(annotations_easy_hard_source, 'r')
    xy_coords_easy_hard  = text_file_easy_hard.readlines()

    text_file_easy = open(annotations_easy_source, 'r')
    xy_coords_easy  = text_file_easy.readlines()

    xy_coords_hard = [coord for coord in xy_coords_easy_hard if coord not in xy_coords_easy ] 
    text_file_hard = open(image_dest +"annotation_hard/" +str(count)+".txt", 'w')
    for coord in xy_coords_hard:
        text_file_hard.write(coord)
    text_file_hard.close()

    annotations_low_motility_source = os.path.join(data_dir, video, minivideo, "annotations_motility_low",
                                           text_file)
    annotations_high_motility_source = os.path.join(data_dir, video, minivideo, "annotations_motility_high",
                                           text_file)
    annotations_mid_motility_source = os.path.join(data_dir, video, minivideo, "annotations_motility_mid",
                                           text_file)
    annotations_wiggle_motility_source = os.path.join(data_dir, video, minivideo, "annotations_motility_wiggle",
                                           text_file)
    
    annotations_sticking_motile_source = os.path.join(data_dir, video, minivideo, "annotations_sticking_motile",
                                           text_file)
    annotations_sticking_stick_source = os.path.join(data_dir, video, minivideo, "annotations_sticking_stick",
                                           text_file)
    annotations_sticking_non_motile_source = os.path.join(data_dir, video, minivideo, "annotations_sticking_non_motile",
                                           text_file)
    
    shutil.copy(annotations_low_motility_source, image_dest +"annotation_motility_low/" +str(count)+".txt")
    shutil.copy(annotations_high_motility_source, image_dest +"annotation_motility_high/" +str(count)+".txt")
    shutil.copy(annotations_mid_motility_source, image_dest +"annotation_motility_mid/" +str(count)+".txt")
    shutil.copy(annotations_wiggle_motility_source, image_dest +"annotation_motility_wiggle/" +str(count)+".txt")
    
    shutil.copy(annotations_sticking_stick_source, image_dest +"annotation_sticking_stick/" +str(count)+".txt")
    shutil.copy(annotations_sticking_motile_source, image_dest +"annotation_sticking_motile/" +str(count)+".txt")
    shutil.copy(annotations_sticking_non_motile_source, image_dest +"annotation_sticking_non_motile/" +str(count)+".txt")
    
    
    if train == True:
        count_train += 1
    else:
        count_test += 1
        
    return count_test, count_train
    

In [14]:
#get backgorund frame for every mini video
#skip the first frame in every mini video
#store image in train images set
#store image+background diff + prev image diff in train images feature set
#similarly for test

def create_data(data_dir, dest_dir, trainfolder, train_video, testfolder, test_video, valfolder, val_video, method="background", num_prev=None, mean=False, sample=False, test_only = False, params=None, optical_flow_prior=1, frame_diff_prior=1):
    os.makedirs(dest_dir, exist_ok=True)
    data_dir_types = ["/images/", "/images_feature/", "/annotation_easy/", "/annotation_hard/", "/annotation_easy_hard/", "/annotation_easy_hard_veryhard/" , "/annotation_veryhard/" , "/annotation_motility_low", "/annotation_motility_wiggle", "/annotation_motility_mid", "/annotation_motility_high", "/annotation_sticking_stick", "/annotation_sticking_motile", "/annotation_sticking_non_motile"]
    for video in test_video:
        for dir_type in data_dir_types:
            os.makedirs(os.path.join(dest_dir, testfolder, video) + dir_type, exist_ok=True)
            
    for video in val_video:
        for dir_type in data_dir_types:
            if test_only == True:
                continue
            os.makedirs(os.path.join(dest_dir, valfolder, video) + dir_type, exist_ok=True)
            
    for dir_type in data_dir_types:
            os.makedirs(os.path.join(dest_dir, testfolder) + dir_type, exist_ok=True)
            if test_only == True:
                continue
            os.makedirs(os.path.join(dest_dir, valfolder) + dir_type, exist_ok=True)
            os.makedirs(dest_dir+trainfolder + dir_type, exist_ok=True)
    
    count_train = 0
    count_test = 0
    count_test_all = 0
    count_val_all = 0

    for video in natsorted(os.listdir(data_dir)):
        
        if not video.startswith('.') and os.path.isdir(os.path.join(data_dir,video))==True:
            if test_only == True and video not in test_video:
                continue
            if video not in train_video + test_video + val_video:
                continue
            for minivideo in natsorted(os.listdir(os.path.join(data_dir,video))) :
                if not minivideo.startswith('.') and os.path.isdir(os.path.join(data_dir,video,minivideo)) == True:
                    video_path = os.path.join(data_dir,video) + "/" + minivideo + "video.mp4"
                    print(video_path)

                    if method == "background":
                        # get the background model
                        background = get_background(video_path, mean, sample=sample)
                        # convert the background model to grayscale format
                        background = cv2.cvtColor(background, cv2.COLOR_BGR2GRAY)
                        skip_frame_list = [0]
                    if method == "nprev":
                        skip_frame_list = list(range(num_prev))
                    if method in ["diff_from_max_absolute_consecutive_frame_diff", "max_absolute_consecutive_frame_diff"]:
                        max_absolute_consecutive_frame_diff = get_absolute_consec_frame_diff_feature(video_path, max_consec_frame_diff=True)
                    if method in ["max_absolute_all_frame_diff"]:
                        absolute_all_frame_diff = get_absolute_all_frame_diff_feature(video_path, max_feature=True)
                    if method in ["min_absolute_all_frame_diff"]:
                        absolute_all_frame_diff = get_absolute_all_frame_diff_feature(video_path, max_feature=False)
                        
                        
                        
                    if video not in (test_video + val_video): 
                        #print(video)
                        train=True
                    else:
                        train=False
                        count_test = 0
                        if video in test_video:
                            testfolder_video = os.path.join(testfolder, video)
                        else:
                            testfolder_video = os.path.join(valfolder, video)
                    
                        
                    if method in ["optical_flow", "optical_flow_median_back", "optical_flow_from_median_frame", 
                                  "optical_flow_combined", "diff_from_max_absolute_consecutive_frame_diff", 
                                  "max_absolute_consecutive_frame_diff", "min_absolute_all_frame_diff", 
                                  "max_absolute_all_frame_diff"]:
                        #print(method)
                        method_flow = cv2.calcOpticalFlowFarneback
                        #params = [0.5, 3, 15, 3, 5, 1.2, 0]  # default Farneback's algorithm parameters
                        # params = [0.5, 4, 18, 3, 5, 1.2, 0]  
                        #params = [0.5, 2, 18, 3, 5, 1.2, 0]  
                        #params = [0.5, 2, 20, 3, 5, 1.2, 0]  
                        
                        background = get_background(video_path, mean, sample=sample)
                        # convert the background model to grayscale format
                        background = cv2.cvtColor(background, cv2.COLOR_BGR2GRAY)
                        
                        if method == "optical_flow_from_median_frame":
                            frames_org, frames_optical_flow = gen_dense_optical_flow_data(method_flow, video_path, params, to_gray=True, median=True, median_frame=background)
                        else :
                            frames_org, frames_optical_flow = gen_dense_optical_flow_data(method_flow, video_path, params, to_gray=True,num_frames_prior=optical_flow_prior)
                            
                        if method == "optical_flow_combined":
                            m_frames_org, m_frames_optical_flow = gen_dense_optical_flow_data(method_flow, video_path, params, to_gray=True, median=True, median_frame=background)
                            c_frames_org, c_frames_optical_flow = gen_dense_optical_flow_data(method_flow, video_path, params, to_gray=True)
    #                     print(len(frames_org))
    #                     print(frames_org[0].shape)
    #                     print(len(frames_optical_flow))
    #                     print(frames_optical_flow[0].shape)
    
                        if method in ["optical_flow", "optical_flow_from_median_frame"]:
                            pure = True
                        else:
                            pure = False
                                              

                        print(len(frames_org))        
                        for i, frame in enumerate(frames_org):
                            #save frame in images
                            if train==True:
                                image_dest = dest_dir + "/" +trainfolder +"/"
                                count = count_train
                            else:
                                #print(video)
                                image_dest = dest_dir +"/" + testfolder_video + "/"
                                count = count_test

                            #print(frame.shape)
                            img = Image.fromarray(frame, "RGB")
                            img.save(image_dest +"images/"+str(count)+".tif")
                            if method == "optical_flow_combined":
                                feature_image = create_feature_image_optical_flow(frame, c_frames_optical_flow[i], pure=pure, optical_flow2=m_frames_optical_flow[i])
                            elif method == "diff_from_max_absolute_consecutive_frame_diff":
                                diff_from_max_absolute_consecutive_frame_diff = get_diff_from_absolute_consec_frame_diff_feature(frame_diff_feature=max_absolute_consecutive_frame_diff, image_path=frame, frame=True)
                                feature_image = create_feature_image_optical_flow(frame, frames_optical_flow[i], pure, diff_from_max_absolute_consecutive_frame_diff, final_channel=True)
                            elif method == "max_absolute_consecutive_frame_diff":
                                feature_image = create_feature_image_optical_flow(frame, frames_optical_flow[i], pure, max_absolute_consecutive_frame_diff, final_channel=False)
                            elif method in ["max_absolute_all_frame_diff", "min_absolute_all_frame_diff"]:
                                feature_image = create_feature_image_optical_flow(frame, frames_optical_flow[i], pure, absolute_all_frame_diff[i+1], final_channel=False)
                            else:
                                feature_image = create_feature_image_optical_flow(frame, frames_optical_flow[i], pure, background)
                            frame_num = i+1
                            
                            count_test, count_train = save_data(image_dest, count, feature_image, frame_num, data_dir, video, minivideo, count_test, 
                                                                                count_train, train)
                            if train == False:
                                if video in test_video:
                                    count = count_test_all
                                    image_dest_all = dest_dir +"/" + testfolder + "/"
                                    count_test_all, count_train = save_data(image_dest_all, count, feature_image, frame_num, data_dir, video, minivideo, count_test_all, 
                                                                                 count_train, train)
                                else:
                                    count = count_val_all
                                    image_dest_all = dest_dir +"/" + valfolder + "/"
                                    count_val_all, count_train = save_data(image_dest_all, count, feature_image, frame_num, data_dir, video, minivideo, count_val_all, 
                                                                                 count_train, train)
                                img.save(image_dest_all +"images/"+str(count)+".tif")
                                
                            
                    else:
                        # print("in else")
                        # print(method)
                        for frame in natsorted(os.listdir(os.path.join(data_dir, video, minivideo, "images"))):
                            frame_num = int(frame.split(".tif")[0])
                            #skip first frame
                            if frame_num not in skip_frame_list:
                                #save frame in images
                                images_source = os.path.join(data_dir, video, minivideo, "images", frame)
                                if train==True:
                                    image_dest = dest_dir + "/" +trainfolder +"/"
                                    count = count_train
                                    #print(count)
                                else:
                                    #print(video)
                                    image_dest = dest_dir +"/" + testfolder_video + "/"
                                    image_dest_all = dest_dir +"/" + testfolder + "/"
                                    count = count_test

                                shutil.copy(images_source, image_dest +"images/"+str(count)+".tif")

                                #create new image
                                prev_frame = str(frame_num-1) +".tif"
                                prev_image = os.path.join(data_dir, video, minivideo, "images", prev_frame) 
                                if method == "background":
                                    feature_image = create_feature_image(image_path=images_source, background=background,prev_image_path=prev_image)
                                if method == "nprev":
                                    background=None
                                    prev_frame2 = str(frame_num-num_prev) +".tif"
                                    prev_image2 = os.path.join(data_dir, video, minivideo, "images", prev_frame2) 
                                    feature_image = create_feature_image(image_path=images_source, background=background, 
                                                                     prev_image_path=prev_image, prev_image_path2=prev_image2)



                                count_test, count_train = save_data(image_dest, count, feature_image, frame_num, data_dir, video, minivideo, count_test, count_train, train)
                                if train == False:
                                    if video in test_video:
                                        count = count_test_all
                                        image_dest_all = dest_dir +"/" + testfolder + "/"
                                        count_test_all, count_train = save_data(image_dest_all, count, feature_image, frame_num, data_dir, video, minivideo, count_test_all, 
                                                                                     count_train, train)
                                    else:
                                        count = count_val_all
                                        image_dest_all = dest_dir +"/" + valfolder + "/"
                                        count_val_all, count_train = save_data(image_dest_all, count, feature_image, frame_num, data_dir, video, minivideo, count_val_all, 
                                                                                     count_train, train)
                                    shutil.copy(images_source, image_dest_all +"images/"+str(count)+".tif")
                                    
                            

# Final data Training with easy and hard 

Create Training Data

In [19]:
target_data_sub_dir = "/data/medha/Bacteria/Data/collagen_motility/"
dest_sub_dir = "/data/medha/Bacteria/DataFeatures/"
exp_name = "collagen_sticking_optical_flow_median_bkg_more_data_2"

In [20]:
data_dir = target_data_sub_dir
dest_dir = os.path.join(dest_sub_dir, exp_name, "data_feature_optical_flow_median_back_2pyr_18win_background_img/")
trainfolder = "train"
testfolder = "test"
valfolder = "val"

#control, control1, 01, 02, 2_1, 2_2, 4_1, 4_2, 5_1, 5_2, 7_1, 7_2
#O_5, 2_6, 4_6, 7_4, 7_7, 7_8

train_video = ["video217", "video218","video222","video223", "video229","video230", "video236", "video237",
               "video242", "video243","video247", "video248", 
               "video226","video234", "video241","video250","video253","video254"]
               
#control_2, 2_3, 4_3, 5_3, 7_3, o_3
val_video = ["video219", "video224", "video231", "video238", "video244", "video249"]

#Control3, O_6, 2_4, 4_4, 7_6, 5_4
test_video = ["video220", "video227", "video232", "video239", "video245", "video252"]

# Control 4, RBS O_4, RBS O_7, RBS 2_5, RBS 2_7, RBS 4_5, RBS 5_5, RBS 7_5, RBS 7_9, RBS 7_10

final_test_options = ["video221", "video225", "video228", "video233", "video235", 
                      "video240", "video246", "video251", "video255", "video256"]
test_video+=final_test_options

params = [0.5, 2, 18, 3, 5, 1.2, 0] 

In [21]:
create_data(data_dir, dest_dir, trainfolder, train_video, testfolder, test_video, valfolder, val_video, 
            method="optical_flow_median_back", params=params)

/data/medha/Bacteria/Data/collagen_motility/video217/frame1video.mp4
150
149
/data/medha/Bacteria/Data/collagen_motility/video218/frame1video.mp4
150
149
/data/medha/Bacteria/Data/collagen_motility/video219/frame1video.mp4
150
149
/data/medha/Bacteria/Data/collagen_motility/video220/frame1video.mp4
150
149
/data/medha/Bacteria/Data/collagen_motility/video221/frame1video.mp4
150
149
/data/medha/Bacteria/Data/collagen_motility/video222/frame1video.mp4
150
149
/data/medha/Bacteria/Data/collagen_motility/video223/frame1video.mp4
150
149
/data/medha/Bacteria/Data/collagen_motility/video224/frame1video.mp4
150
149
/data/medha/Bacteria/Data/collagen_motility/video225/frame1video.mp4
150
149
/data/medha/Bacteria/Data/collagen_motility/video226/frame1video.mp4
150
149
/data/medha/Bacteria/Data/collagen_motility/video227/frame1video.mp4
150
149
/data/medha/Bacteria/Data/collagen_motility/video228/frame1video.mp4
150
149
/data/medha/Bacteria/Data/collagen_motility/video229/frame1video.mp4
150
149

In [18]:
# Test Data Video Specific
# Optical Flow + median background  frame diff 34k 0.00125lr [0.5, 2, 18, 3, 5, 1.2, 0] 
# 0.93
# 0.84

#test_video_main = test_video

#test_video_main = [f"video{video_num}"]#["video3", "video4","video7", "video9","video8", "video10"]
test_video_main = test_video + final_test_options

for video in test_video_main:
    print(video)
    data_dir = target_data_sub_dir
    dest_dir = os.path.join(dest_sub_dir, exp_name , f"data_{video}_feature_optical_flow_median_back_2pyr_18win_background_img/")
    trainfolder = "train"
    testfolder = "test"
    valfolder = "val"
    val_video = []
    test_video = [f"{video}"]
    params = [0.5, 2, 18, 3, 5, 1.2, 0]  

    create_data(data_dir, dest_dir, trainfolder, train_video, testfolder, test_video, valfolder, val_video, 
                method="optical_flow_median_back", test_only = True, params=params)

video220
/data/medha/Bacteria/Data/collagen_motility/video220/frame1video.mp4
150
149
video227
/data/medha/Bacteria/Data/collagen_motility/video227/frame1video.mp4
150
149
video232
/data/medha/Bacteria/Data/collagen_motility/video232/frame1video.mp4
150
149
video239
/data/medha/Bacteria/Data/collagen_motility/video239/frame1video.mp4
150
149
video245
/data/medha/Bacteria/Data/collagen_motility/video245/frame1video.mp4
150
149
video252
/data/medha/Bacteria/Data/collagen_motility/video252/frame1video.mp4
150
149
video221
/data/medha/Bacteria/Data/collagen_motility/video221/frame1video.mp4
150
149
video225
/data/medha/Bacteria/Data/collagen_motility/video225/frame1video.mp4
150
149
video228
/data/medha/Bacteria/Data/collagen_motility/video228/frame1video.mp4
150
149
video233
/data/medha/Bacteria/Data/collagen_motility/video233/frame1video.mp4
150
149
video235
/data/medha/Bacteria/Data/collagen_motility/video235/frame1video.mp4
150
149
video240
/data/medha/Bacteria/Data/collagen_motility/v

In [16]:
target_data_sub_dir = "/data/medha/Bacteria/Data/collagen_motility/"
dest_sub_dir = "/data/medha/Bacteria/DataFeatures/"
exp_name = "collagen_motility_optical_flow_median_bkg_more_data"

In [17]:
data_dir = target_data_sub_dir
dest_dir = os.path.join(dest_sub_dir, exp_name, "data_feature_optical_flow_median_back_2pyr_18win_background_img/")
trainfolder = "train"
testfolder = "test"
valfolder = "val"

#control, control1, 01, 02, 2_1, 2_1, 4_1, 4_2, 5_1, 5_2, 
train_video = ["video139", "video140","video143","video144", "video150","video151", "video157", "video158",
               "video163", "video164","video172", "video173", 
               "video147","video155","video157","video162","video174","video168","video169"]
               
#control_2, 2_3, 4_3, 5_3, 7_3, o_3
val_video = ["video175", "video145", "video152", "video176", "video159", "video165"]

#Control3, O_6, 2_4, 4_4, 7_6, 5_4
test_video = ["video141", "video148", "video153", "video160", "video167", "video170"]

# Control 4, RBS O_4, RBS O_9, RBS 2_5, RBS 2_7, RBS 4_5, RBS 5_5, RBS 7_5, RBS 7_9, RBS 7_10

final_test_options = ["video142", "video146", "video149", "video154", "video156", 
                      "video161", "video171", "video166", "video214", "video215"]


params = [0.5, 2, 18, 3, 5, 1.2, 0] 

In [18]:
#optical_flow_median_back orginial
#optical flow median back with optical_Flow_prior is optical flow from xth previous frame
# "diff_from_max_absolute_consecutive_frame_diff" creates a feature for diff from the "max consecutive frame diff" feature, with a frame diff prior for xth frame diff

In [19]:
create_data(data_dir, dest_dir, trainfolder, train_video, testfolder, test_video, valfolder, val_video, 
            method="optical_flow_median_back", params=params)

/data/medha/Bacteria/Data/collagen_motility/video139/frame1video.mp4
150
149
/data/medha/Bacteria/Data/collagen_motility/video140/frame1video.mp4
150
149
/data/medha/Bacteria/Data/collagen_motility/video141/frame1video.mp4
150
149
/data/medha/Bacteria/Data/collagen_motility/video143/frame1video.mp4
150
149
/data/medha/Bacteria/Data/collagen_motility/video144/frame1video.mp4
150
149
/data/medha/Bacteria/Data/collagen_motility/video145/frame1video.mp4
150
149
/data/medha/Bacteria/Data/collagen_motility/video147/frame1video.mp4
150
149
/data/medha/Bacteria/Data/collagen_motility/video148/frame1video.mp4
150
149
/data/medha/Bacteria/Data/collagen_motility/video150/frame1video.mp4
150
149
/data/medha/Bacteria/Data/collagen_motility/video151/frame1video.mp4
150
149
/data/medha/Bacteria/Data/collagen_motility/video152/frame1video.mp4
150
149
/data/medha/Bacteria/Data/collagen_motility/video153/frame1video.mp4
150
149
/data/medha/Bacteria/Data/collagen_motility/video155/frame1video.mp4
150
149

In [21]:
# Test Data Video Specific
# Optical Flow + median background  frame diff 34k 0.00125lr [0.5, 2, 18, 3, 5, 1.2, 0] 
# 0.93
# 0.84

#test_video_main = test_video

#test_video_main = [f"video{video_num}"]#["video3", "video4","video7", "video9","video8", "video10"]
test_video_main = ["video141", "video148", "video153", "video160", "video167", "video170"]
#test_video_main = final_test_options

for video in test_video_main:
    print(video)
    data_dir = target_data_sub_dir
    dest_dir = os.path.join(dest_sub_dir, exp_name , f"data_{video}_feature_optical_flow_median_back_2pyr_18win_background_img/")
    trainfolder = "train"
    testfolder = "test"
    valfolder = "val"
    val_video = []
    test_video = [f"{video}"]
    params = [0.5, 2, 18, 3, 5, 1.2, 0]  

    create_data(data_dir, dest_dir, trainfolder, train_video, testfolder, test_video, valfolder, val_video, 
                method="optical_flow_median_back", test_only = True, params=params)

video141
/data/medha/Bacteria/Data/collagen_motility/video141/frame1video.mp4
150
149
video148
/data/medha/Bacteria/Data/collagen_motility/video148/frame1video.mp4
150
149
video153
/data/medha/Bacteria/Data/collagen_motility/video153/frame1video.mp4
150
149
video160
/data/medha/Bacteria/Data/collagen_motility/video160/frame1video.mp4
150
149
video167
/data/medha/Bacteria/Data/collagen_motility/video167/frame1video.mp4
150
149
video170
/data/medha/Bacteria/Data/collagen_motility/video170/frame1video.mp4
150
149


In [16]:
target_data_sub_dir = "/data/medha/Bacteria/Data/collagen_motility/"
dest_sub_dir = "/data/medha/Bacteria/DataFeatures/"
exp_name = "collagen_motility_sticking_optical_flow_median_bkg_more_data"

In [17]:
data_dir = target_data_sub_dir
dest_dir = os.path.join(dest_sub_dir, exp_name, "data_feature_optical_flow_median_back_2pyr_18win_background_img/")
trainfolder = "train"
testfolder = "test"
valfolder = "val"

#control, control1, 01, 02, 2_1, 2_2, 4_1, 4_2, 5_1, 5_2, 7_1, 7_2
train_video = ["video177", "video178","video182","video183", "video189","video190", "video196", "video197",
               "video202", "video203","video206", "video207", 
               "video186","video194","video195","video201","video209","video212","video213"]
               
#control_2, 2_3, 4_3, 5_3, 7_3, o_3
val_video = ["video179", "video184", "video191", "video198", "video204", "video208"]

#Control3, O_6, 2_4, 4_4, 7_6, 5_4
test_video = ["video180", "video187", "video192", "video199", "video216", "video211"]

#control 4, O_4, O_7,   2_5, 4_5, 5_5, 7_5
# Final_test = = ["video181", "video185", “video188”, "video193", "video200", "video205", "video210"]


params = [0.5, 2, 18, 3, 5, 1.2, 0] 

In [18]:
create_data(data_dir, dest_dir, trainfolder, train_video, testfolder, test_video, valfolder, val_video, 
            method="optical_flow_median_back", params=params)

/data/medha/Bacteria/Data/collagen_motility/video177/frame1video.mp4
150
149
/data/medha/Bacteria/Data/collagen_motility/video178/frame1video.mp4
150
149
/data/medha/Bacteria/Data/collagen_motility/video179/frame1video.mp4
150
149
/data/medha/Bacteria/Data/collagen_motility/video180/frame1video.mp4
150
149
/data/medha/Bacteria/Data/collagen_motility/video182/frame1video.mp4
150
149
/data/medha/Bacteria/Data/collagen_motility/video183/frame1video.mp4
150
149
/data/medha/Bacteria/Data/collagen_motility/video184/frame1video.mp4
150
149
/data/medha/Bacteria/Data/collagen_motility/video186/frame1video.mp4
150
149
/data/medha/Bacteria/Data/collagen_motility/video187/frame1video.mp4
150
149
/data/medha/Bacteria/Data/collagen_motility/video189/frame1video.mp4
150
149
/data/medha/Bacteria/Data/collagen_motility/video190/frame1video.mp4
150
149
/data/medha/Bacteria/Data/collagen_motility/video191/frame1video.mp4
150
149
/data/medha/Bacteria/Data/collagen_motility/video192/frame1video.mp4
150
149

In [19]:
# Test Data Video Specific
# Optical Flow + median background  frame diff 34k 0.00125lr [0.5, 2, 18, 3, 5, 1.2, 0] 
# 0.93
# 0.84

#test_video_main = test_video

#test_video_main = [f"video{video_num}"]#["video3", "video4","video7", "video9","video8", "video10"]
test_video_main = ["video180", "video187", "video192", "video199", "video216", "video211"]

for video in test_video_main:
    print(video)
    data_dir = target_data_sub_dir
    dest_dir = os.path.join(dest_sub_dir, exp_name , f"data_{video}_feature_optical_flow_median_back_2pyr_18win_background_img/")
    trainfolder = "train"
    testfolder = "test"
    valfolder = "val"
    val_video = []
    test_video = [f"{video}"]
    params = [0.5, 2, 18, 3, 5, 1.2, 0]  

    create_data(data_dir, dest_dir, trainfolder, train_video, testfolder, test_video, valfolder, val_video, 
                method="optical_flow_median_back", test_only = True, params=params)

video180
/data/medha/Bacteria/Data/collagen_motility/video180/frame1video.mp4
150
149
video187
/data/medha/Bacteria/Data/collagen_motility/video187/frame1video.mp4
150
149
video192
/data/medha/Bacteria/Data/collagen_motility/video192/frame1video.mp4
150
149
video199
/data/medha/Bacteria/Data/collagen_motility/video199/frame1video.mp4
150
149
video216
/data/medha/Bacteria/Data/collagen_motility/video216/frame1video.mp4
150
149
video211
/data/medha/Bacteria/Data/collagen_motility/video211/frame1video.mp4
150
149


In [17]:
target_data_sub_dir = "/data/medha/Bacteria/Data/liquid/"
dest_sub_dir = "/data/medha/Bacteria/DataFeatures/"
exp_name = "liquid_all_tracked"

In [18]:
test_video_main = ["video257", "video258"]

for video in test_video_main:
    print(video)
    data_dir = target_data_sub_dir
    dest_dir = os.path.join(dest_sub_dir, exp_name , f"data_{video}_feature_optical_flow_median_back_2pyr_18win_background_img/")
    trainfolder = "train"
    testfolder = "test"
    valfolder = "val"
    val_video = []
    test_video = [f"{video}"]
    params = [0.5, 2, 18, 3, 5, 1.2, 0]  

    create_data(data_dir, dest_dir, trainfolder, train_video, testfolder, test_video, valfolder, val_video, 
                method="optical_flow_median_back", test_only = True, params=params)

video257
/data/medha/Bacteria/Data/liquid/video257/frame1video.mp4
100
99
video258
/data/medha/Bacteria/Data/liquid/video258/frame1video.mp4
100
99


### Original split used for training and tetsing in workshop paper-combined

In [54]:
target_data_sub_dir = "../Data/data_vh/"
dest_sub_dir = "../DataFeatures/"
exp_name = "agar0.2_combined_test"

In [16]:
data_dir = target_data_sub_dir
dest_dir = os.path.join(dest_sub_dir, exp_name, "data_feature_optical_flow_median_back_2pyr_18win_background_img/")
trainfolder = "train"
testfolder = "test"
valfolder = "val"

#Train: 11, 13, 14, 15, 16, 17, 28, 31, 32, 36, 37, 38
train_video = ["video11", "video13","video14","video15", "video16","video17", "video28", "video31",
              "video32", "video36", "video37", "video38"]

#Val: 12, 18, 20, 21, 23, 26,

val_video = ["video12", "video18", "video20", "video21", "video23", "video26"]

#8,9,19,22,24,25,27,29,30,33,34,35
test_video = ["video85", "video24", "video30", "video35", "video83", "video84"]

params = [0.5, 2, 18, 3, 5, 1.2, 0] 

#create_data(data_dir, dest_dir, trainfolder, train_video, testfolder, test_video, valfolder, val_video, method="optical_flow_median_back", params=params)

In [55]:
# Test Data Video Specific
# Optical Flow + median background  frame diff 34k 0.00125lr [0.5, 2, 18, 3, 5, 1.2, 0] 
# 0.93
# 0.84

#test_video_main = test_video

#test_video_main = [f"video{video_num}"]#["video3", "video4","video7", "video9","video8", "video10"]
test_video_main = test_video = ["video19", "video22", "video29", "video30", "video83", "video84"] #["video87"] #["video86", "video87", "video88", "video89"]

for video in test_video_main:
    print(video)
    data_dir = target_data_sub_dir
    dest_dir = os.path.join(dest_sub_dir, exp_name , f"data_{video}_feature_optical_flow_median_back_2pyr_18win_background_img/")
    trainfolder = "train"
    testfolder = "test"
    valfolder = "val"
    val_video = []
    test_video = [f"{video}"]
    params = [0.5, 2, 18, 3, 5, 1.2, 0]  

    create_data(data_dir, dest_dir, trainfolder, train_video, testfolder, test_video, valfolder, val_video, method="optical_flow_median_back", test_only = True, params=params)

video19
../Data/data_vh/video19/frame1video.mp4
100
99
video22
../Data/data_vh/video22/frame1video.mp4
100
99
video29
../Data/data_vh/video29/frame1video.mp4
100
99
video30
../Data/data_vh/video30/frame1video.mp4
100
99
video83
../Data/data_vh/video83/frame1video.mp4
100
99
video84
../Data/data_vh/video84/frame1video.mp4
100
99


In [67]:
# Test Data Video Specific
# Optical Flow + median background  frame diff 34k 0.00125lr [0.5, 2, 18, 3, 5, 1.2, 0] 
# 0.93
# 0.84

#test_video_main = test_video
target_data_sub_dir = "../Data/data_vh/"
dest_sub_dir = "../DataFeatures/"
exp_name = "random"
#test_video_main = [f"video{video_num}"]#["video3", "video4","video7", "video9","video8", "video10"]
test_video_main = test_video = ["video29"]#["video19", "video22", "video29", "video30", "video83", "video84"] #["video87"] #["video86", "video87", "video88", "video89"]
train_video = ["video29"]
for video in test_video_main:
    print(video)
    data_dir = target_data_sub_dir
    dest_dir = os.path.join(dest_sub_dir, exp_name , f"data_{video}_feature_optical_flow_median_back_2pyr_18win_background_img/")
    trainfolder = "train"
    testfolder = "test"
    valfolder = "val"
    val_video = []
    test_video = [f"{video}"]
    params = [0.5, 2, 18, 3, 5, 1.2, 0]  

    create_data(data_dir, dest_dir, trainfolder, train_video, testfolder, test_video, valfolder, val_video, method="optical_flow_median_back", test_only = True, params=params)

video29
../Data/data_vh/video29/frame1video.mp4
100
99


In [15]:
target_data_sub_dir = "/alldata/medha/CleanCodeData/Data/liquid/"
dest_sub_dir = "/alldata/medha/CleanCodeData/DataFeatures/"
exp_name = "liquid_all_tracked/"

In [16]:
# Test Data Video Specific
# Optical Flow + median background  frame diff 34k 0.00125lr [0.5, 2, 18, 3, 5, 1.2, 0] 


test_video_main = ["video257", "video258"] 
train_video=[]
for video in test_video_main:
    print(video)
    data_dir = target_data_sub_dir
    dest_dir = os.path.join(dest_sub_dir, exp_name , f"data_{video}_feature_optical_flow_median_back_2pyr_18win_background_img/")
    trainfolder = "train"
    testfolder = "test"
    valfolder = "val"
    val_video = []
    test_video = [f"{video}"]
    params = [0.5, 2, 18, 3, 5, 1.2, 0]  

    create_data(data_dir, dest_dir, trainfolder, train_video, testfolder, test_video, valfolder, val_video, method="optical_flow_median_back", test_only = True, params=params)

video257


PermissionError: [Errno 13] Permission denied: '/alldata'